In [1]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt 
from sklearn.kernel_ridge import KernelRidge
import seaborn as sns
from tensorflow.keras import datasets, layers, models
base_path = os.getcwd()

/home/kamesh-k/anaconda3/envs/RDKit/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kamesh-k/anaconda3/envs/RDKit/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kamesh-k/anaconda3/envs/RDKit/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kamesh-k/anaconda3/envs/RDKit/lib/pyt

In [2]:
train_data = pd.read_csv("Train_Data.csv")
train_data.head()

,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,CV,Structure
0,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,C
1,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,O
2,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,C#C
3,285.48839,38.982300,34.298920,2.1089,14.18,-0.2670,-0.0406,0.2263,59.9891,0.026603,-114.483613,-114.480746,-114.479802,-114.505268,6.413,C=O
4,80.46225,19.906490,19.906330,0.0000,23.95,-0.3385,0.1041,0.4426,109.5031,0.074542,-79.764152,-79.760666,-79.759722,-79.787269,10.098,CC


In [3]:
structure = train_data['Structure']
superset = ''.join(structure)
encoders = list(set(superset))

In [4]:
features = dict(zip(encoders,np.arange(len(encoders))))
max_length = 0 
len_encoders = len(encoders)
for struct in structure:
    max_length = max(len(struct),max_length)
max_size = len(train_data)
representation = np.zeros((max_size,max_length,len(encoders)))
for index in range(max_size):
    struct = structure[index]
    j = 0
    for i in struct:
        representation[index,j,features[i]]=1
        j = j + 1

In [5]:
train_size = 2000/len(train_data)
data_y = train_data['CV']
scaler_y = MinMaxScaler()
data_y = scaler_y.fit_transform(np.array(train_data['CV']).reshape(-1, 1))
train_X, test_X, train_Y, test_Y = train_test_split(representation,data_y,train_size=train_size)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(max_length, (3, 2), activation='relu', input_shape=(max_length,len_encoders,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 1), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(1,activation='linear'))
model.compile(
  optimizer='adam',
  loss='mean_squared_error',
  metrics=['accuracy'],
)
train_X = train_X.reshape(-1,max_length,len_encoders,1)
test_X = test_X.reshape(-1,max_length,len_encoders,1)
history = model.fit(
  train_X,
  train_Y,
  epochs=50,
  validation_data = (test_X,test_Y)
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 2000 samples, validate on 54773 samples
Epoch 1/50
2000/2000 [==============================] - 8s 4ms/sample - loss: 0.0330 - acc: 0.0000e+00 - val_loss: 0.0053 - val_acc: 3.6514e-05
Epoch 2/50
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.0093 - acc: 0.0000e+00 - val_loss: 0.0041 - val_acc: 3.6514e-05
Epoch 3/50
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.0078 - acc: 0.0000e+00 - val_loss: 0.0036 - val_acc: 3.6514e-05
Epoch 4/50
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0065 - acc: 0.0000e+00 - val_loss: 0.0032 - val_acc: 3.6514e-05
Epoch 5/50
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.0060 - acc: 0.0000e+00 - val_loss: 0.0029 - val_acc: 3.6514e-05
Epoch 6/50
2000/2000 [==============================] - 7s 4ms/sample - loss: 0.0053 - acc: 0.0000e+00 - val_loss

0.7937395492958079

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 10, 22)        154       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 5, 22)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 3, 64)          12736     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 1, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 1, 64)          12352     
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               3

In [29]:
transfer_model = model
print("Number of layers in the base model: ", len(transfer_model.layers))

# Freeze all the layers before the `fine_tune_at` layer
fine_tune_at = 6
for layer in transfer_model.layers[:fine_tune_at]:
  layer.trainable =  False

transfer_model.summary()

Number of layers in the base model:  11
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 10, 22)        154       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 5, 22)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 3, 64)          12736     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 1, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 1, 64)          12352     
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)   

In [30]:
transfer_model.compile(
  optimizer='adam',
  loss='mean_squared_error',
  metrics=['accuracy'],
)

train_size = 2000/len(train_data)
data_y = train_data['gap']
scaler_y = MinMaxScaler()
data_y = scaler_y.fit_transform(np.array(train_data['gap']).reshape(-1, 1))
train_X, test_X, train_Y, test_Y = train_test_split(representation,data_y,train_size=train_size)

train_X = train_X.reshape(-1,max_length,len_encoders,1)
test_X = test_X.reshape(-1,max_length,len_encoders,1)
history = model.fit(
  train_X,
  train_Y,
  epochs=50,
  validation_data = (test_X,test_Y)
)

Train on 2000 samples, validate on 54773 samples
Epoch 1/50
2000/2000 [==============================] - 7s 4ms/sample - loss: 0.0031 - acc: 0.0000e+00 - val_loss: 0.0016 - val_acc: 3.6514e-05
Epoch 2/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0015 - acc: 0.0000e+00 - val_loss: 0.0013 - val_acc: 3.6514e-05
Epoch 3/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0012 - val_acc: 3.6514e-05
Epoch 4/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0011 - val_acc: 3.6514e-05
Epoch 5/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0013 - acc: 0.0000e+00 - val_loss: 0.0011 - val_acc: 3.6514e-05
Epoch 6/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0012 - acc: 0.0000e+00 - val_loss: 0.0011 - val_acc: 3.6514e-05
Epoch 7/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0011 - 

In [37]:
model_alpha = models.Sequential()
model_alpha.add(layers.Conv2D(max_length, (3, 2), activation='relu', input_shape=(max_length,len_encoders,1)))
model_alpha.add(layers.MaxPooling2D((2, 2)))
model_alpha.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_alpha.add(layers.MaxPooling2D((2, 2)))
model_alpha.add(layers.Conv2D(64, (3, 1), activation='relu'))
model_alpha.add(layers.Flatten())
model_alpha.add(layers.Dense(256,activation='relu'))
model_alpha.add(Dropout(0.2))
model_alpha.add(layers.Dense(128,activation='relu'))
model_alpha.add(Dropout(0.2))
model_alpha.add(layers.Dense(1,activation='linear'))
model_alpha.compile(
  optimizer='adam',
  loss='mean_squared_error',
  metrics=['accuracy'],
)
train_size = 2000/len(train_data)
data_y = train_data['gap']
scaler_y = MinMaxScaler()
data_y = scaler_y.fit_transform(np.array(train_data['gap']).reshape(-1, 1))
train_X, test_X, train_Y, test_Y = train_test_split(representation,data_y,train_size=train_size)

train_X = train_X.reshape(-1,max_length,len_encoders,1)
test_X = test_X.reshape(-1,max_length,len_encoders,1)
history = model_alpha.fit(
  train_X,
  train_Y,
  epochs=50,
  validation_data = (test_X,test_Y)
)

Train on 2000 samples, validate on 54773 samples
Epoch 1/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0065 - acc: 0.0000e+00 - val_loss: 0.0059 - val_acc: 1.8257e-05
Epoch 2/50
2000/2000 [==============================] - 7s 4ms/sample - loss: 0.0058 - acc: 0.0000e+00 - val_loss: 0.0059 - val_acc: 1.8257e-05
Epoch 3/50
2000/2000 [==============================] - 7s 4ms/sample - loss: 0.0055 - acc: 0.0000e+00 - val_loss: 0.0057 - val_acc: 1.8257e-05
Epoch 4/50
2000/2000 [==============================] - 8s 4ms/sample - loss: 0.0052 - acc: 0.0000e+00 - val_loss: 0.0056 - val_acc: 1.8257e-05
Epoch 5/50
2000/2000 [==============================] - 8s 4ms/sample - loss: 0.0049 - acc: 0.0000e+00 - val_loss: 0.0057 - val_acc: 1.8257e-05
Epoch 6/50
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.0046 - acc: 0.0000e+00 - val_loss: 0.0057 - val_acc: 1.8257e-05
Epoch 7/50
2000/2000 [==============================] - 7s 4ms/sample - loss: 0.0045 - 

In [38]:
from sklearn.metrics import r2_score
r2_score(model.predict(test_X),test_Y)

0.35705409819234024